<a href="https://colab.research.google.com/github/shumshersubashgautam/Pyspark-NLP/blob/main/Albert_model_for_HuggingFace_Spark_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***ALBERT models from HuggingFace ***

In [50]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 66.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.13.1 re

In [51]:
from transformers import AlbertTokenizer, TFAlbertModel
import tensorflow as tf

# albert-base-v2
MODEL_NAME = 'albert-base-v2'

AlbertTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFAlbertModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFAlbertModel.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

try downloading TF weights


Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [52]:
!ls -l {MODEL_NAME}

total 45684
-rw-r--r-- 1 root root      792 May  5 04:06 config.json
drwxr-xr-x 3 root root     4096 May  5 04:06 saved_model
-rw-r--r-- 1 root root 46771352 May  5 04:06 tf_model.h5


In [53]:
!ls -l {MODEL_NAME}/saved_model/1

total 10016
drwxr-xr-x 2 root root     4096 May  5 04:06 assets
-rw-r--r-- 1 root root       53 May  5 04:06 fingerprint.pb
-rw-r--r-- 1 root root    24297 May  5 04:06 keras_metadata.pb
-rw-r--r-- 1 root root 10217113 May  5 04:06 saved_model.pb
drwxr-xr-x 2 root root     4096 May  5 04:06 variables


In [54]:
!ls -l {MODEL_NAME}_tokenizer

total 752
-rw-r--r-- 1 root root    286 May  5 04:05 special_tokens_map.json
-rw-r--r-- 1 root root 760289 May  5 04:05 spiece.model
-rw-r--r-- 1 root root    577 May  5 04:05 tokenizer_config.json


In [55]:
# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {MODEL_NAME}/saved_model/1/assets

# **Importing and Saving ALBERT in Spark NLP**

In [56]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 4.4.1
setup Colab for PySpark 3.2.3 and Spark NLP 4.4.1


In [57]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

In [58]:
from sparknlp.annotator import *

albert = AlbertEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)\
 .setDimension(768)\
 .setStorageRef('albert_base_uncased') 

In [59]:
albert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

In [60]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

In [61]:
! ls -l {MODEL_NAME}_spark_nlp

total 56348
-rw-r--r-- 1 root root   760289 May  5 04:06 albert_spp
-rw-r--r-- 1 root root 56926612 May  5 04:06 albert_tensorflow
drwxr-xr-x 3 root root     4096 May  5 04:06 fields
drwxr-xr-x 2 root root     4096 May  5 04:06 metadata


In [62]:
albert_loaded = AlbertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

In [63]:
albert_loaded.getStorageRef()

'albert_base_uncased'

# **AlbertForQuestionAnswering**

In [64]:
from transformers import TFAlbertForQuestionAnswering, AlbertTokenizer 
import tensorflow as tf

MODEL_NAME = 'twmkn9/albert-base-v2-squad2'

tokenizer = AlbertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFAlbertForQuestionAnswering.from_pretrained(MODEL_NAME)
except:
  model = TFAlbertForQuestionAnswering.from_pretrained(MODEL_NAME, from_pt=True)


# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

All PyTorch model weights were used when initializing TFAlbertForQuestionAnswering.

All the weights of TFAlbertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForQuestionAnswering for predictions without further training.


In [65]:
!ls -l {MODEL_NAME}

total 43388
-rw-r--r-- 1 root root      844 May  5 04:08 config.json
drwxr-xr-x 3 root root     4096 May  5 04:08 saved_model
-rw-r--r-- 1 root root 44417688 May  5 04:08 tf_model.h5


In [66]:
!ls -l {MODEL_NAME}/saved_model/1

total 10304
drwxr-xr-x 2 root root     4096 May  5 04:08 assets
-rw-r--r-- 1 root root       55 May  5 04:08 fingerprint.pb
-rw-r--r-- 1 root root    24517 May  5 04:08 keras_metadata.pb
-rw-r--r-- 1 root root 10512298 May  5 04:08 saved_model.pb
drwxr-xr-x 2 root root     4096 May  5 04:08 variables


In [67]:
!ls -l {MODEL_NAME}_tokenizer

total 752
-rw-r--r-- 1 root root    173 May  5 04:08 special_tokens_map.json
-rw-r--r-- 1 root root 760289 May  5 04:08 spiece.model
-rw-r--r-- 1 root root    722 May  5 04:08 tokenizer_config.json


In [68]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/spiece.model {asset_path}

In [69]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 744
-rw-r--r-- 1 root root 760289 May  5 04:09 spiece.model


# **Save AlbertForQuestionAnswering in Spark NLP**

In [70]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 4.4.1
setup Colab for PySpark 3.2.3 and Spark NLP 4.4.1


In [71]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

In [72]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = AlbertForQuestionAnswering.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

In [73]:
spanClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

In [74]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

In [75]:
! ls -l {MODEL_NAME}_spark_nlp

total 54332
-rw-r--r-- 1 root root 54863622 May  5 04:10 albert_classification_tensorflow
-rw-r--r-- 1 root root   760289 May  5 04:10 albert_spp
drwxr-xr-x 3 root root     4096 May  5 04:10 fields
drwxr-xr-x 2 root root     4096 May  5 04:10 metadata


In [76]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = AlbertForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



# **AlbertForSequenceClassification models from HuggingFace**

In [77]:
from transformers import TFAlbertForSequenceClassification, AlbertTokenizer 
import tensorflow as tf

MODEL_NAME = 'mohsenfayyaz/albert-base-v2-toxicity'

tokenizer = AlbertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFAlbertForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFAlbertForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

try downloading TF weights
try downloading PyTorch weights


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertForSequenceClassification: ['albert.embeddings.position_ids']
- This IS expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForSequenceClassification for predictions without further training.


In [78]:
!ls -l {MODEL_NAME}

total 45696
-rw-r--r-- 1 root root      914 May  5 04:13 config.json
drwxr-xr-x 3 root root     4096 May  5 04:13 saved_model
-rw-r--r-- 1 root root 46781688 May  5 04:13 tf_model.h5


In [79]:
!ls -l {MODEL_NAME}/saved_model/1

total 10388
drwxr-xr-x 2 root root     4096 May  5 04:13 assets
-rw-r--r-- 1 root root       55 May  5 04:13 fingerprint.pb
-rw-r--r-- 1 root root    25986 May  5 04:13 keras_metadata.pb
-rw-r--r-- 1 root root 10596112 May  5 04:13 saved_model.pb
drwxr-xr-x 2 root root     4096 May  5 04:13 variables


In [80]:
!ls -l {MODEL_NAME}_tokenizer

total 752
-rw-r--r-- 1 root root    286 May  5 04:13 special_tokens_map.json
-rw-r--r-- 1 root root 760289 May  5 04:13 spiece.model
-rw-r--r-- 1 root root    572 May  5 04:13 tokenizer_config.json


In [81]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {asset_path}

In [82]:
# get label2id dictionary 
labels = model.config.id2label
# sort the dictionary based on the id
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [83]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 748
-rw-r--r-- 1 root root     15 May  5 04:13 labels.txt
-rw-r--r-- 1 root root 760289 May  5 04:13 spiece.model


**Save AlbertForSequenceClassification in Spark NLP**

In [84]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-05-05 04:13:53--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-05-05 04:13:53--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               Installing PySpark 3.2.3 and Spark NLP 4.4.1
setup Colab for PySpark 3.2.3 an

In [85]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

In [86]:
from sparknlp.annotator import *

sequenceClassifier = AlbertForSequenceClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(128)

In [87]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

In [88]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

In [89]:
! ls -l {MODEL_NAME}_spark_nlp

total 56720
-rw-r--r-- 1 root root 57308312 May  5 04:14 albert_classification_tensorflow
-rw-r--r-- 1 root root   760289 May  5 04:14 albert_spp
drwxr-xr-x 4 root root     4096 May  5 04:14 fields
drwxr-xr-x 2 root root     4096 May  5 04:14 metadata


In [90]:
sequenceClassifier_loaded = AlbertForSequenceClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

In [91]:
# .getClasses was introduced in spark-nlp==3.4.0
sequenceClassifier_loaded.getClasses()

['Toxic', 'Non-Toxic']

In [92]:
from pyspark.ml import Pipeline

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier_loaded    
])

# couple of simple examples
example = spark.createDataFrame([["I love you!"], ['I feel lucky to be here.'], ['I hate her!']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show()

+--------------------+-----------+
|                text|     result|
+--------------------+-----------+
|         I love you!|[Non-Toxic]|
|I feel lucky to b...|[Non-Toxic]|
|         I hate her!|    [Toxic]|
+--------------------+-----------+



# **ALBERT For TOken Classification**

In [93]:
from transformers import TFAlbertForTokenClassification, AlbertTokenizer 
import tensorflow as tf

MODEL_NAME = 'HooshvareLab/albert-fa-zwnj-base-v2-ner'

tokenizer = AlbertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFAlbertForTokenClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFAlbertForTokenClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

try downloading TF weights


Some layers from the model checkpoint at HooshvareLab/albert-fa-zwnj-base-v2-ner were not used when initializing TFAlbertForTokenClassification: ['dropout_4']
- This IS expected if you are initializing TFAlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertForTokenClassification were initialized from the model checkpoint at HooshvareLab/albert-fa-zwnj-base-v2-ner.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForTokenClassification for predictions without further training.


In [94]:
!ls -l {MODEL_NAME}

total 43444
-rw-r--r-- 1 root root     1630 May  5 04:16 config.json
drwxr-xr-x 3 root root     4096 May  5 04:16 saved_model
-rw-r--r-- 1 root root 44476132 May  5 04:16 tf_model.h5


In [95]:
!ls -l {MODEL_NAME}/saved_model/1

total 10344
drwxr-xr-x 2 root root     4096 May  5 04:16 assets
-rw-r--r-- 1 root root       54 May  5 04:16 fingerprint.pb
-rw-r--r-- 1 root root    26535 May  5 04:16 keras_metadata.pb
-rw-r--r-- 1 root root 10549790 May  5 04:16 saved_model.pb
drwxr-xr-x 2 root root     4096 May  5 04:16 variables


In [96]:
!ls -l {MODEL_NAME}_tokenizer

total 848
-rw-r--r-- 1 root root    173 May  5 04:16 special_tokens_map.json
-rw-r--r-- 1 root root 857476 May  5 04:16 spiece.model
-rw-r--r-- 1 root root    576 May  5 04:16 tokenizer_config.json


In [97]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {asset_path}

In [98]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [99]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 844
-rw-r--r-- 1 root root    121 May  5 04:16 labels.txt
-rw-r--r-- 1 root root 857476 May  5 04:16 spiece.model


# **Import and Save AlbertForTokenClassification in Spark NLP**

In [100]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 4.4.1
setup Colab for PySpark 3.2.3 and Spark NLP 4.4.1


In [101]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

In [102]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = AlbertForTokenClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(128)

In [103]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

In [104]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

In [105]:
! ls -l {MODEL_NAME}_spark_nlp

total 54524
-rw-r--r-- 1 root root 54958174 May  5 04:17 albert_classification_tensorflow
-rw-r--r-- 1 root root   857476 May  5 04:17 albert_spp
drwxr-xr-x 4 root root     4096 May  5 04:17 fields
drwxr-xr-x 2 root root     4096 May  5 04:17 metadata


In [106]:
tokenClassifier_loaded = AlbertForTokenClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

In [107]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['I-PCT',
 'B-PRO',
 'I-EVE',
 'B-LOC',
 'I-ORG',
 'B-FAC',
 'B-EVE',
 'B-TIM',
 'I-DAT',
 'B-MON',
 'B-PCT',
 'I-MON',
 'I-LOC',
 'I-FAC',
 'I-PRO',
 'I-TIM',
 'I-PER',
 'B-DAT',
 'B-ORG',
 'O',
 'B-PER']

In [108]:
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded    
])

# couple of simple examples
example = spark.createDataFrame([["این سریال به صورت رسمی در تاریخ دهم می ۲۰۱۱ توسط شبکه فاکس برای پخش رزرو شد."], 
                                 ['دفتر مرکزی شرکت کامیکو در شهر ساسکاتون ساسکاچوان قرار دارد.'], 
                                 ['در سال ۲۰۱۳ درگذشت و آندرتیکر و کین برای او مراسم یادبود گرفتند.']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|این سریال به صورت...|[O, O, O, O, O, O...|
|دفتر مرکزی شرکت ک...|[O, O, B-ORG, I-O...|
|در سال ۲۰۱۳ درگذش...|[O, B-DAT, I-DAT,...|
+--------------------+--------------------+

